### documents의 메타 정보 추출 (제목, 요약, 키워드)

In [ ]:
import transformers
import torch

model_id = "rtzr/ko-gemma-2-9b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [2]:
def request_prompt(message, max_new_tokens=100):
    with torch.no_grad():  # cuda 메모리 확보하기 위해 
        try:
            prompt = pipeline.tokenizer.apply_chat_template(
                message, 
                tokenize=False, 
                add_generation_prompt=True
            )

            terminators = [
                pipeline.tokenizer.eos_token_id,
                pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
            ]

            outputs = pipeline(
                prompt,
                max_new_tokens=max_new_tokens,
                eos_token_id=terminators,
                do_sample=False,
            )
            return outputs[0]["generated_text"][len(prompt):]
        except Exception as e:
            print(e)
            return ""

In [ ]:
# persona_meta = """당신은 과학 기술에 특화된 상식 RAG 시스템입니다.
# 입력된 문서의 색인을 위한 title, summary, keyword를 한글로 생성하세요. 
# 응답의 상세한 조건은 다음을 따르세요:
# **출력**: JSON 포맷으로 작성해주세요.
#    - 예시
#    {
#       "title": "YF 백신: 노란열 바이러스 예방을 위한 생백신",
#       "summary": "YF 백신은 파스퇴르 통로에서 개발된 약독화 생 바이러스 백신으로, 노란열 바이러스에 대한 면역력을 강화하여 예방합니다. 노란열은 모기에 의해 전파되며 중증 증상을 유발할 수 있는 바이러스입니다. YF 백신은 전 세계적으로 안전하고 효과적인 예방 수단으로 사용됩니다.",
#       "keyword": ["YF 백신", "노란열 바이러스", "예방 백신", "파스퇴르 통로", "약독화 생 바이러스"]
#    }
# **주의사항**
#    - 응답은 반드시 예시처럼 JSON Body 만 출력하세요.
# """
# print(persona_meta)

In [ ]:
persona_meta = """당신은 과학 기술에 특화된 상식 RAG 시스템입니다.
다음은 과학 기술 문서의 한국어 번역본입니다. 색인을 위한 title, summary, keywords를 생성하세요. 
응답의 상세한 조건은 다음을 따르세요:
**필드**:
-title: 문서의 내용을 잘 나타내는 간결하고 명확한 제목을 생성해주세요.
-summary: 문서의 핵심 내용을 요약해주세요. 전문 용어를 사용해도 좋지만, 필요한 경우 간단한 설명을 덧붙여주세요.
-keywords: 문서의 핵심 주제와 개념을 나타내는 키워드를 추출해주세요. 해당 분야의 문서를 검색하거나 분류할 때 자주 사용되는 것이어야 합니다. 특정 단어는 한글과 영어를 모두 포함해주세요.
**출력**: JSON 포맷으로 작성해주세요.
   - 예시
   {
      "title": "YF 백신: 노란열 바이러스 예방을 위한 생백신",
      "summary": "YF 백신은 파스퇴르 통로에서 개발된 약독화 생 바이러스 백신으로, 노란열 바이러스에 대한 면역력을 강화하여 예방합니다. 노란열은 모기에 의해 전파되며 중증 증상을 유발할 수 있는 바이러스입니다. YF 백신은 전 세계적으로 안전하고 효과적인 예방 수단으로 사용됩니다.",
      "keywords": ["YF 백신", "노란열 바이러스", "예방 백신", "파스퇴르 통로", "약독화 생 바이러스"]
   }
**주의사항**
   - 응답은 반드시 예시처럼 JSON Body 만 출력하세요.
"""
print(persona_meta)

In [16]:
import re
import json

def extract_fields(response):
    # 정규 표현식을 사용하여 응답에서 JSON 부분만 추출
    json_pattern = r'\{.*?\}'
    match = re.search(json_pattern, response, re.DOTALL)
    
    try:
        # JSON 파싱 시도
        json_str = match.group(0)  # JSON 문자열 추출
        result = json.loads(json_str)
    except json.JSONDecodeError as e:
        # 파싱 실패 시 에러 위치 출력 및 처리
        print(f"JSON 파싱 실패({e.pos}): {e},")
    except Exception as e:
        print(f"No Json Format: {e}")

    if result:
        return result
    
    try:
        # title 추출
        title_match = re.search(r'"title"\s*:\s*"([^"]*)"', response)
        title = title_match.group(1) if title_match else None
        
        # summary 추출
        summary_match = re.search(r'"summary"\s*:\s*"([^"]*)"', response)
        summary = summary_match.group(1) if summary_match else None
        
        # keyword 추출 (배열 형태로 추출)
        keyword_match = re.search(r'"keyword"\s*:\s*\[([^\]]*)\]', response)
        if keyword_match:
            keyword_string = keyword_match.group(1)
            keywords = re.findall(r'"([^"]*)"', keyword_string)
        else:
            keywords = None

        result = {
            "title": title,
            "summary": summary,
            "keywords": keywords
            }
        return json.loads(result)

    except Exception as e:
        print(f"Error occurred: {e}")
        raise e

In [19]:
import json
import gc

def generate_meta_data(input_filename, output_filename):
    with open(input_filename) as f, open(output_filename, "w") as of:
        idx = 0

        for line in f:
            print(f'[{idx}]')
            # if idx > 5:
            #   break
            
            j = json.loads(line)
            document = j["content"]
            print(document)
            prompt = [{"role": "system", "content": persona_meta}] + [{"role": "user", "content": document}]
            # print(f'prompt : {prompt}')
            response = request_prompt(prompt, 800)
            print(f'plain : {response}')

            try:
                data = extract_fields(response)
                print(f'{json.dumps(data, ensure_ascii=False)}')

                # 수정된 JSON을 출력 파일에 저장
                of.write(f'{json.dumps(data, ensure_ascii=False)}\n')

            except Exception as e:
                id = j["docid"]
                print(f"++ Error! {id} : {e}")

            idx += 1

            del response
            gc.collect()
            torch.cuda.empty_cache()

In [ ]:
generate_meta_data("./data/documents.jsonl", "./data/documents_meta_키워드강화.jsonl")